In [ ]:

## REF: https://www.kaggle.com/code/dansbecker/getting-started-with-sql-and-bigquery 



In [ ]:
https://www.kaggle.com/code/beerus/exercise-getting-started-with-sql-and-bigquery/edit 
    

In [ ]:


from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "global_air_quality" table
client.list_rows(table, max_results=5).to_dataframe()




In [ ]:

# list of tables in the dataset: 
tables = list(client.list_tables(dataset))

for itm in tables:
    #* "table_id" 
    print(itm.table_id)



In [ ]:
# Query to select countries with units of "ppm"
first_query = """
SELECT country 
FROM `bigquery-public-data.openaq.global_air_quality`
WHERE unit = 'ppm'
""" # Your code goes here

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()

# View top few rows of results
print(first_results.head())

# Check your answer
q_1.check()





In [ ]:
# Your code goes here
country_spend_pct_query = """
                          SELECT country_name, AVG(value) AS avg_ed_spending_pct 
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year <= 2017 AND year >= 2010
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

# Check your answer
q_1.check()



In [ ]:
# Your code goes here
code_count_query = """
SELECT indicator_code, indicator_name, COUNT(1) AS num_rows    
FROM `bigquery-public-data.world_bank_intl_education.international_education`
WHERE year = 2016
GROUP BY indicator_code, indicator_name 
HAVING num_rows >= 175
ORDER BY num_rows DESC 

"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())

# Check your answer
q_2.check()



In [ ]:
##** GOOD ONE  #* CTE

# Your code goes here
speeds_query = """
               WITH RelevantRides AS
               (
                   SELECT trip_start_timestamp, trip_miles, trip_seconds
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01' AND trip_seconds > 0 AND trip_miles > 0
               )
               SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour_of_day, COUNT(1) AS num_trips, 3600 * SUM(trip_miles)/SUM(trip_seconds) AS avg_mph  
               FROM RelevantRides
               GROUP BY hour_of_day
               ORDER BY hour_of_day
               """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
speeds_query_job = client.query(speeds_query, job_config=safe_config) # Your code here

# API request - run the query, and return a pandas DataFrame
speeds_result = speeds_query_job.to_dataframe() # Your code here

# View results
print(speeds_result)

# Check your answer
q_5.check()




In [ ]:
# Your code here
bigquery_experts_query = """
SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q 
ON a.parent_id = q.id
WHERE q.tags LIKE '%bigquery%'
GROUP BY a.owner_user_id 
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check() 


In [ ]:

##** IMP: **passing a variable in {} braces**
#----------
# convert what you've done to a general function a website could call on the backend to get experts on any topic: 

def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%{topic}%'   #**
               GROUP BY a.owner_user_id
               """

    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results




In [ ]:

## NOTES: 
#========

#** If you are ever unsure what to put inside the COUNT() function, you can do COUNT(1) to count the rows in each group.

#** It also scans less data than if supplied column names (making it faster and using less of your data access quota). 

#* Note that because it tells SQL how to apply aggregate functions (like COUNT()), it doesn't make sense to 
    # use GROUP BY without an aggregate function. Similarly, if you have any GROUP BY clause, then all variables must be passed to either a
        #* GROUP BY command, or
        #* an aggregation function.

#** this query won't work, because the author column isn't passed to an aggregate function or a GROUP BY clause:
query_bad = """
            SELECT author, parent, COUNT(id)
            FROM `bigquery-public-data.hacker_news.comments`
            GROUP BY parent
            """

#** ORDER BY is usually the last clause in your query, and it sorts the results returned by the rest of your query. 
#*  The ORDER BY clause also works for columns containing text, where the results show up in alphabetical order.

#** AS is more powerful when combined with WITH in what's called a "common table expression" (CTE). 
#** A common table expression (or CTE) is a temporary table that you return within your query. 
    # CTEs are helpful for splitting your queries into readable chunks, and you can write queries against them.
    #* CTEs only exist inside the query where you create them, and you can't reference them in later queries.
    #* CTE query is always broken into two parts: (1) first, we create the CTE, and then (2) we write a query that uses the CTE.
    
#* In the JOIN query, ON determines which column in each table to use to combine the tables.     
    

    

